<a href="https://colab.research.google.com/github/SkySssw/Coursera_Capstone/blob/main/The_Battle_of_Neighborhoods_Home_Prices_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The Battle of Neighborhoods

# The Battle of Neighborhoods


## Introduction to business problem






Nowaday, the home price in each big cities are rising. So, I would like to know how relation between venues and home price in the city. I choose Toronto and Montreal which are big cities in Canada in this situation.

## The data and how it will be used to solve the problem

To analyze this problem , I needed home prices data and location data. I obtain location data by using  'Geospatial_Coordinates.csv' form last project to get Toronto coordinates. For Montreal coordinates, I download csv file from '//simplemaps.com/data/canada-cities' and I used Forsquare API to get venues of Toronto and Montreal.
 

In [2]:
!pip install beautifulsoup4
!pip3 install wikipedia
!apt-get install python3-bs4 (for Python 3)
!pip install geocoder
import geocoder # import geocoder
import requests as rq #library to handle requests
import pandas as pd
import numpy as np
import wikipedia as wiki
from bs4 import BeautifulSoup #library to parse html doc
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('download completed!')

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11697 sha256=c47a8d547f3eacfed0efdc3cefeb45c3549651be56be947f1631c3edce54ca36
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `apt-get install python3-bs4 (for Python 3)'
     |████████████████████████████████| 102kB 6.0MB/s 
download completed!


In [4]:
housing = pd.read_csv('/content/wellbeing-toronto-housing-excel2.csv')
housing.head()

,Neighbourhood,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
0,West Humber-Clairville,317508.0,690.0,1810.0,6.8,5.3,38.0,0.31,952.0,557.0
1,Mount Olive-Silverstone-Jamestown,251119.0,4110.0,13395.0,42.9,40.9,41.0,7.00,1146.0,1049.0
2,Thistletown-Beaumond Heights,414216.0,430.0,1260.0,13.4,12.4,7.0,1.33,447.0,270.0
3,Rexdale-Kipling,392271.0,600.0,1050.0,15.2,10.0,19.0,4.20,432.0,186.0
4,Elms-Old Rexdale,233832.0,870.0,2305.0,27.3,24.1,14.0,1.00,463.0,239.0


In [5]:
housing = housing.rename(columns={'Neighbourhood':'Neighborhood'})
housing

,Neighborhood,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
0,West Humber-Clairville,317508.0,690.0,1810.0,6.8,5.3,38.0,0.31,952.0,557.0
1,Mount Olive-Silverstone-Jamestown,251119.0,4110.0,13395.0,42.9,40.9,41.0,7.00,1146.0,1049.0
2,Thistletown-Beaumond Heights,414216.0,430.0,1260.0,13.4,12.4,7.0,1.33,447.0,270.0
3,Rexdale-Kipling,392271.0,600.0,1050.0,15.2,10.0,19.0,4.20,432.0,186.0
4,Elms-Old Rexdale,233832.0,870.0,2305.0,27.3,24.1,14.0,1.00,463.0,239.0
...,...,...,...,...,...,...,...,...,...,...
135,West Hill,308229.0,3010.0,5880.0,31.1,22.1,22.0,8.40,2089.0,725.0
136,Woburn,316584.0,5090.0,14870.0,27.8,27.9,26.0,4.91,2660.0,1306.0
137,Eglinton East,274020.0,3935.0,10105.0,48.8,44.3,33.0,2.59,865.0,688.0
138,Scarborough Village,356096.0,2905.0,7860.0,49.2,47.3,22.0,4.40,922.0,683.0


In [6]:
urk_wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table_class = "wikitable sortable jquery-tablesorter"
response = rq.get(urk_wiki)
print(response.status_code) # check if it shows '200', so it's legal to use data

200


In [7]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')

In [8]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto_data = df
toronto_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Use Geospatial Coordinates file from previous project.

In [10]:
geo = pd.read_csv('/content/Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
geo['PostalCode'] = geo['Postal Code']

In [12]:
toronto_data_merge = toronto_data.merge(geo,how='left',on='PostalCode')
toronto_data_merge

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y,43.636258,-79.498509


In [13]:
toronto_data_merge=toronto_data_merge.drop(['Postal Code','PostalCode'], axis= 1)


In [14]:
toronto_data_merge.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [15]:
Neighborhood_new = []
for x in toronto_data_merge['Neighborhood']:
    temp = x.split(',')
    Neighborhood_new.append(temp[0])
Neighborhood_new

['Parkwoods',
 'Victoria Village',
 'Regent Park',
 'Lawrence Manor',
 'Ontario Provincial Government',
 'Islington Avenue',
 'Malvern',
 'Don Mills North',
 'Parkview Hill',
 'Garden District',
 'Glencairn',
 'West Deane Park',
 'Rouge Hill',
 'Don Mills South',
 'Woodbine Heights',
 'St. James Town',
 'Humewood-Cedarvale',
 'Eringate',
 'Guildwood',
 'The Beaches',
 'Berczy Park',
 'Caledonia-Fairbanks',
 'Woburn',
 'Leaside',
 'Central Bay Street',
 'Christie',
 'Cedarbrae',
 'Hillcrest Village',
 'Bathurst Manor',
 'Thorncliffe Park',
 'Richmond',
 'Dufferin',
 'Scarborough Village',
 'Fairview',
 'Northwood Park',
 'The Danforth  East',
 'Harbourfront East',
 'Little Portugal',
 'Kennedy Park',
 'Bayview Village',
 'Downsview East',
 'The Danforth West',
 'Toronto Dominion Centre',
 'Brockton',
 'Golden Mile',
 'York Mills',
 'Downsview West',
 'India Bazaar',
 'Commerce Court',
 'North Park',
 'Humber Summit',
 'Cliffside',
 'Willowdale',
 'Downsview Central',
 'Studio District',

In [16]:
toronto_data_merge['Neighborhood'] = Neighborhood_new
toronto_data_merge.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,Regent Park,43.654260,-79.360636
3,North York,Lawrence Manor,43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [17]:
toronto_data_merge.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,Regent Park,43.654260,-79.360636
3,North York,Lawrence Manor,43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [18]:
housing

,Neighborhood,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
0,West Humber-Clairville,317508.0,690.0,1810.0,6.8,5.3,38.0,0.31,952.0,557.0
1,Mount Olive-Silverstone-Jamestown,251119.0,4110.0,13395.0,42.9,40.9,41.0,7.00,1146.0,1049.0
2,Thistletown-Beaumond Heights,414216.0,430.0,1260.0,13.4,12.4,7.0,1.33,447.0,270.0
3,Rexdale-Kipling,392271.0,600.0,1050.0,15.2,10.0,19.0,4.20,432.0,186.0
4,Elms-Old Rexdale,233832.0,870.0,2305.0,27.3,24.1,14.0,1.00,463.0,239.0
...,...,...,...,...,...,...,...,...,...,...
135,West Hill,308229.0,3010.0,5880.0,31.1,22.1,22.0,8.40,2089.0,725.0
136,Woburn,316584.0,5090.0,14870.0,27.8,27.9,26.0,4.91,2660.0,1306.0
137,Eglinton East,274020.0,3935.0,10105.0,48.8,44.3,33.0,2.59,865.0,688.0
138,Scarborough Village,356096.0,2905.0,7860.0,49.2,47.3,22.0,4.40,922.0,683.0


In [19]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 10 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Neighborhood                             140 non-null    object 
 1   Home Prices                              140 non-null    float64
 2   Mid-Century Highrise Households          140 non-null    float64
 3   Mid-Century Highrise Population          140 non-null    float64
 4   Percent Mid-Century Highrise Households  140 non-null    float64
 5   Percent Mid-Century Highrise Population  140 non-null    float64
 6   Rent Bank Applicants                     140 non-null    float64
 7   Social Housing Turnover                  140 non-null    float64
 8   Social Housing Units                     140 non-null    float64
 9   Social Housing Waiting List              140 non-null    float64
dtypes: float64(9), object(1)
memory usage: 11.1+ KB


Merging data to have a Home prices data for each neighborhood.


In [20]:
toronto_housing = toronto_data_merge.merge(housing,how='left',on='Neighborhood')
toronto_housing

,Borough,Neighborhood,Latitude,Longitude,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
0,North York,Parkwoods,43.753259,-79.329656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,North York,Victoria Village,43.725882,-79.315572,365107.0,2345.0,5410.0,32.2,31.5,15.0,7.08,891.0,459.0
2,Downtown Toronto,Regent Park,43.654260,-79.360636,484444.0,1190.0,2655.0,30.9,26.5,4.0,7.25,2591.0,650.0
3,North York,Lawrence Manor,43.718518,-79.464763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,Etobicoke,The Kingsway,43.653654,-79.506944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,East Toronto Business,Enclave of M4L,43.662744,-79.321558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,Etobicoke,Old Mill South,43.636258,-79.498509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
toronto_housing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 13 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Borough                                  103 non-null    object 
 1   Neighborhood                             103 non-null    object 
 2   Latitude                                 103 non-null    float64
 3   Longitude                                103 non-null    float64
 4   Home Prices                              21 non-null     float64
 5   Mid-Century Highrise Households          21 non-null     float64
 6   Mid-Century Highrise Population          21 non-null     float64
 7   Percent Mid-Century Highrise Households  21 non-null     float64
 8   Percent Mid-Century Highrise Population  21 non-null     float64
 9   Rent Bank Applicants                     21 non-null     float64
 10  Social Housing Turnover                  21 non-nu

In [22]:
toronto_housing = toronto_housing.fillna(0)

In [23]:
toronto_housing.head()

,Borough,Neighborhood,Latitude,Longitude,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
0,North York,Parkwoods,43.753259,-79.329656,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,North York,Victoria Village,43.725882,-79.315572,365107.0,2345.0,5410.0,32.2,31.5,15.0,7.08,891.0,459.0
2,Downtown Toronto,Regent Park,43.654260,-79.360636,484444.0,1190.0,2655.0,30.9,26.5,4.0,7.25,2591.0,650.0
3,North York,Lawrence Manor,43.718518,-79.464763,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0


Select column which Home Prices are more than zero.


In [24]:
toronto_housing_final=toronto_housing[toronto_housing['Home Prices']>0]

toronto_housing_final

,Borough,Neighborhood,Latitude,Longitude,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
1,North York,Victoria Village,43.725882,-79.315572,365107.0,2345.0,5410.0,32.2,31.5,15.0,7.08,891.0,459.0
2,Downtown Toronto,Regent Park,43.654260,-79.360636,484444.0,1190.0,2655.0,30.9,26.5,4.0,7.25,2591.0,650.0
6,Scarborough,Malvern,43.806686,-79.194353,256488.0,1060.0,3175.0,7.9,7.0,32.0,1.17,927.0,851.0
16,York,Humewood-Cedarvale,43.693781,-79.428191,800442.0,1410.0,2390.0,21.4,16.9,8.0,0.38,445.0,247.0
18,Scarborough,Guildwood,43.763573,-79.188711,444309.0,605.0,1020.0,15.3,10.4,3.0,0.00,0.0,62.0
19,East Toronto,The Beaches,43.676357,-79.293031,751945.0,150.0,220.0,1.6,1.0,17.0,0.33,193.0,143.0
22,Scarborough,Woburn,43.770992,-79.216917,316584.0,5090.0,14870.0,27.8,27.9,26.0,4.91,2660.0,1306.0
27,North York,Hillcrest Village,43.803762,-79.363452,484743.0,1500.0,2690.0,22.9,15.2,0.0,6.88,1028.0,252.0
28,North York,Bathurst Manor,43.754328,-79.442259,568877.0,1230.0,2625.0,20.3,17.0,6.0,0.50,385.0,232.0
29,East York,Thorncliffe Park,43.705369,-79.349372,313618.0,5110.0,14785.0,76.1,76.9,11.0,1.44,551.0,1105.0


In [25]:
toronto_housing_final.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
0,North York,Victoria Village,43.725882,-79.315572,365107.0,2345.0,5410.0,32.2,31.5,15.0,7.08,891.0,459.0
1,Downtown Toronto,Regent Park,43.654260,-79.360636,484444.0,1190.0,2655.0,30.9,26.5,4.0,7.25,2591.0,650.0
2,Scarborough,Malvern,43.806686,-79.194353,256488.0,1060.0,3175.0,7.9,7.0,32.0,1.17,927.0,851.0
3,York,Humewood-Cedarvale,43.693781,-79.428191,800442.0,1410.0,2390.0,21.4,16.9,8.0,0.38,445.0,247.0
4,Scarborough,Guildwood,43.763573,-79.188711,444309.0,605.0,1020.0,15.3,10.4,3.0,0.00,0.0,62.0
5,East Toronto,The Beaches,43.676357,-79.293031,751945.0,150.0,220.0,1.6,1.0,17.0,0.33,193.0,143.0
6,Scarborough,Woburn,43.770992,-79.216917,316584.0,5090.0,14870.0,27.8,27.9,26.0,4.91,2660.0,1306.0
7,North York,Hillcrest Village,43.803762,-79.363452,484743.0,1500.0,2690.0,22.9,15.2,0.0,6.88,1028.0,252.0
8,North York,Bathurst Manor,43.754328,-79.442259,568877.0,1230.0,2625.0,20.3,17.0,6.0,0.50,385.0,232.0
9,East York,Thorncliffe Park,43.705369,-79.349372,313618.0,5110.0,14785.0,76.1,76.9,11.0,1.44,551.0,1105.0


In [26]:
toronto_housing_final = toronto_housing_final.reset_index(drop=True)

In [27]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

/bin/bash: conda: command not found


In [28]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.65238435, -79.38356765.


In [29]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [30]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_housing_final['Latitude'], toronto_housing_final['Longitude'], toronto_housing_final['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [31]:
CLIENT_ID = '5WW4JYUW5SMINYV0A2B0AWIBNOOF13K3MV4JS5SBHKZQVZCN' # your Foursquare ID
CLIENT_SECRET = 'UB2Z34VR2VZCWIE3WQXFY3BYHL3MUTPRJRRSR3AHGNMU1RSX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5WW4JYUW5SMINYV0A2B0AWIBNOOF13K3MV4JS5SBHKZQVZCN
CLIENT_SECRET:UB2Z34VR2VZCWIE3WQXFY3BYHL3MUTPRJRRSR3AHGNMU1RSX


In [32]:
toronto_housing_final.loc[0,"Neighborhood"]

'Victoria Village'

In [33]:
neighborhood_latitude = toronto_housing_final.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_housing_final.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_housing_final.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Victoria Village are 43.725882299999995, -79.31557159999998.


In [34]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5WW4JYUW5SMINYV0A2B0AWIBNOOF13K3MV4JS5SBHKZQVZCN&client_secret=UB2Z34VR2VZCWIE3WQXFY3BYHL3MUTPRJRRSR3AHGNMU1RSX&v=20180605&ll=43.725882299999995,-79.31557159999998&radius=500&limit=100'

In [35]:
results = rq.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60e3083c890aaa1091821846'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4c633acb86b6be9a61268e34-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_hockey_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d185941735',
         'name': 'Hockey Arena',
         'pluralName': 'Hockey Arenas',
         'primary': True,
         'shortName': 'Hockey'}],
       'id': '4c633acb86b6be9a61268e34',
       'location': {'cc': 'CA',
        'country': 'Canada',
        'distance': 267,
        'formattedAddress': ['Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72348055545508,
          'lng': -79.31563520925143}],
        'lat': 43.72348055545508,
        'lng': -79.3156352092

In [36]:
import json

In [37]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Portugril,Portuguese Restaurant,43.725819,-79.312785
2,Tim Hortons,Coffee Shop,43.725517,-79.313103
3,Pizza Nova,Pizza Place,43.725824,-79.312860


In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = rq.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
toronto_venues = getNearbyVenues(names=toronto_housing_final['Neighborhood'],
                                   latitudes=toronto_housing_final['Latitude'],
                                   longitudes=toronto_housing_final['Longitude']
                                  )

Victoria Village
Regent Park
Malvern
Humewood-Cedarvale
Guildwood
The Beaches
Woburn
Hillcrest Village
Bathurst Manor
Thorncliffe Park
Scarborough Village
Little Portugal
Kennedy Park
Bayview Village
Humber Summit
Weston
Dorset Park
Willowdale West
Milliken
New Toronto
Alderwood


In [42]:
print(toronto_venues.shape)
toronto_venues.head()

(208, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
1,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
2,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
3,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
4,Regent Park,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


In [43]:
toronto_venues_home_price = toronto_venues.merge(toronto_housing_final,how='left',on='Neighborhood')
toronto_venues_home_price

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Latitude,Longitude,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
0,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena,North York,43.725882,-79.315572,365107.0,2345.0,5410.0,32.2,31.5,15.0,7.08,891.0,459.0
1,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant,North York,43.725882,-79.315572,365107.0,2345.0,5410.0,32.2,31.5,15.0,7.08,891.0,459.0
2,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop,North York,43.725882,-79.315572,365107.0,2345.0,5410.0,32.2,31.5,15.0,7.08,891.0,459.0
3,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place,North York,43.725882,-79.315572,365107.0,2345.0,5410.0,32.2,31.5,15.0,7.08,891.0,459.0
4,Regent Park,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,Downtown Toronto,43.654260,-79.360636,484444.0,1190.0,2655.0,30.9,26.5,4.0,7.25,2591.0,650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Alderwood,43.602414,-79.543484,Tim Hortons,43.602396,-79.545048,Coffee Shop,Etobicoke,43.602414,-79.543484,504233.0,350.0,630.0,7.5,5.3,2.0,2.50,84.0,69.0
204,Alderwood,43.602414,-79.543484,Pizza Pizza,43.605340,-79.547252,Pizza Place,Etobicoke,43.602414,-79.543484,504233.0,350.0,630.0,7.5,5.3,2.0,2.50,84.0,69.0
205,Alderwood,43.602414,-79.543484,Rexall,43.601951,-79.545694,Pharmacy,Etobicoke,43.602414,-79.543484,504233.0,350.0,630.0,7.5,5.3,2.0,2.50,84.0,69.0
206,Alderwood,43.602414,-79.543484,Taras Shevchenko Ukrainian Community Centre,43.602207,-79.543662,Dance Studio,Etobicoke,43.602414,-79.543484,504233.0,350.0,630.0,7.5,5.3,2.0,2.50,84.0,69.0


In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alderwood,8,8,8,8,8,8
Bathurst Manor,22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
Dorset Park,5,5,5,5,5,5
Guildwood,9,9,9,9,9,9
Hillcrest Village,6,6,6,6,6,6
Humber Summit,1,1,1,1,1,1
Humewood-Cedarvale,4,4,4,4,4,4
Kennedy Park,6,6,6,6,6,6


In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 97 uniques categories.


In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Yoga Studio,Antique Shop,Art Gallery,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beer Store,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Café,Chinese Restaurant,Chocolate Shop,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Field,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,...,Japanese Restaurant,Juice Bar,Korean BBQ Restaurant,Korean Restaurant,Liquor Store,Malay Restaurant,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Neighborhood,New American Restaurant,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Record Shop,Rental Car Location,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Spa,Supermarket,Sushi Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,Alderwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
204,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,Alderwood,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
205,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,Alderwood,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
206,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,Alderwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
toronto_onehot.shape

(208, 97)

In [48]:
toronto_onehot_home_price = toronto_onehot.merge(toronto_housing_final[['Neighborhood','Home Prices']], how='left',on='Neighborhood')
toronto_onehot_home_price

,Yoga Studio,Antique Shop,Art Gallery,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beer Store,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Café,Chinese Restaurant,Chocolate Shop,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Field,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,...,Juice Bar,Korean BBQ Restaurant,Korean Restaurant,Liquor Store,Malay Restaurant,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Neighborhood,New American Restaurant,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Record Shop,Rental Car Location,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Spa,Supermarket,Sushi Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Home Prices
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,484444.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,Alderwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,504233.0
204,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,Alderwood,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,504233.0
205,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,Alderwood,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,504233.0
206,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,Alderwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,504233.0


In [49]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Antique Shop,Art Gallery,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beer Store,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Café,Chinese Restaurant,Chocolate Shop,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Field,French Restaurant,Fried Chicken Joint,...,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean BBQ Restaurant,Korean Restaurant,Liquor Store,Malay Restaurant,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,New American Restaurant,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Record Shop,Rental Car Location,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Spa,Supermarket,Sushi Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop
0,Alderwood,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.125000,0.000000,0.000000,0.00000,0.00000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0,0.125000,0.250000,0.000000,0.000000,0.00,0.125000,0.00000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000
1,Bathurst Manor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.00000,0.045455,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.090909,0.000000,0.000000,0.00000,0.00000,0.000,0.045455,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.045455,...,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.045455,0.00000,0.045455,0.00000,0.045455,0.000000,0.0,0.045455,0.045455,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.045455,0.045455,0.000000,0.045455,0.000000,0.045455,0.045455,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000
2,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.250000,0.25,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00000,0.25000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000
3,Dorset Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.20,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.00000,0.000000
4,Guildwood,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111

In [50]:
address = 'Quebec, QC'

geolocator = Nominatim(user_agent="qc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montreal are 46.8259601, -71.2352226.


In [51]:
#Montreal Location
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url_qc = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url_qc # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5WW4JYUW5SMINYV0A2B0AWIBNOOF13K3MV4JS5SBHKZQVZCN&client_secret=UB2Z34VR2VZCWIE3WQXFY3BYHL3MUTPRJRRSR3AHGNMU1RSX&v=20180605&ll=43.725882299999995,-79.31557159999998&radius=500&limit=100'

In [52]:
results_qc = rq.get(url_qc).json()
results_qc

{'meta': {'code': 200, 'requestId': '60e3087c8492d9498305afe5'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4c633acb86b6be9a61268e34-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_hockey_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d185941735',
         'name': 'Hockey Arena',
         'pluralName': 'Hockey Arenas',
         'primary': True,
         'shortName': 'Hockey'}],
       'id': '4c633acb86b6be9a61268e34',
       'location': {'cc': 'CA',
        'country': 'Canada',
        'distance': 267,
        'formattedAddress': ['Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72348055545508,
          'lng': -79.31563520925143}],
        'lat': 43.72348055545508,
        'lng': -79.3156352092

In [53]:
venues_qc = results_qc['response']['groups'][0]['items']
    
nearby_venues_qc = pd.json_normalize(venues_qc) # flatten JSON

# filter columns
filtered_columns_qc = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_qc =nearby_venues_qc.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_qc['venue.categories'] = nearby_venues_qc.apply(get_category_type, axis=1)

# clean columns
nearby_venues_qc.columns = [col.split(".")[-1] for col in nearby_venues_qc.columns]

nearby_venues_qc

,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Portugril,Portuguese Restaurant,43.725819,-79.312785
2,Tim Hortons,Coffee Shop,43.725517,-79.313103
3,Pizza Nova,Pizza Place,43.725824,-79.312860


In [54]:
print('{} venues were returned by Foursquare.'.format(nearby_venues_qc.shape[0]))

4 venues were returned by Foursquare.


In [56]:
# https://simplemaps.com/data/canada-cities

cnd = pd.read_csv('/content/canadacities.csv')
cnd.head()

,city,city_ascii,province_id,province_name,lat,lng,population,density,timezone,ranking,postal,id
0,Toronto,Toronto,ON,Ontario,43.7417,-79.3733,5429524.0,4334.4,America/Toronto,1,M5T M5V M5P M5S M5R M5E M5G M5A M5C M5B M5M M5...,1124279679
1,Montréal,Montreal,QC,Quebec,45.5089,-73.5617,3519595.0,3889.0,America/Montreal,1,H1X H1Y H1Z H1P H1R H1S H1T H1V H1W H1H H1J H1...,1124586170
2,Vancouver,Vancouver,BC,British Columbia,49.2500,-123.1000,2264823.0,5492.6,America/Vancouver,1,V6Z V6S V6R V6P V6N V6M V6L V6K V6J V6H V6G V6...,1124825478
3,Calgary,Calgary,AB,Alberta,51.0500,-114.0667,1239220.0,1501.1,America/Edmonton,1,T1Y T2H T2K T2J T2L T2N T2A T2C T2B T2E T2G T2...,1124690423
4,Edmonton,Edmonton,AB,Alberta,53.5344,-113.4903,1062643.0,1360.9,America/Edmonton,1,T5X T5Y T5Z T5P T5R T5S T5T T5V T5W T5H T5J T5...,1124290735


In [57]:
cnd_qc = cnd[cnd['province_name']=='Quebec']
cnd_qc.head()

,city,city_ascii,province_id,province_name,lat,lng,population,density,timezone,ranking,postal,id
1,Montréal,Montreal,QC,Quebec,45.5089,-73.5617,3519595.0,3889.0,America/Montreal,1,H1X H1Y H1Z H1P H1R H1S H1T H1V H1W H1H H1J H1...,1124586170
8,Quebec City,Quebec City,QC,Quebec,46.8139,-71.2081,705103.0,1173.2,America/Montreal,1,G1N G1M G1L G1K G1J G1H G1G G1E G1C G1B G1Y G1...,1124823933
13,Laval,Laval,QC,Quebec,45.5833,-73.7500,422993.0,1710.9,America/Montreal,2,H7N H7L H7M H7J H7K H7H H7G H7E H7B H7C H7A H7...,1124922301
21,Gatineau,Gatineau,QC,Quebec,45.4833,-75.6500,276245.0,773.7,America/Montreal,2,J8P J8R J8T J8Y J8X J8Z J8M J9J J9H J9A,1124722129
24,Longueuil,Longueuil,QC,Quebec,45.5333,-73.5167,239700.0,2002.0,America/Montreal,2,J4T J4V J4P J4R J4M J4L J4N J4H J4K J4J J4G J3...,1124122753


In [58]:
cnd_qc.reset_index(drop=True)

,city,city_ascii,province_id,province_name,lat,lng,population,density,timezone,ranking,postal,id
0,Montréal,Montreal,QC,Quebec,45.5089,-73.5617,3519595.0,3889.0,America/Montreal,1,H1X H1Y H1Z H1P H1R H1S H1T H1V H1W H1H H1J H1...,1124586170
1,Quebec City,Quebec City,QC,Quebec,46.8139,-71.2081,705103.0,1173.2,America/Montreal,1,G1N G1M G1L G1K G1J G1H G1G G1E G1C G1B G1Y G1...,1124823933
2,Laval,Laval,QC,Quebec,45.5833,-73.7500,422993.0,1710.9,America/Montreal,2,H7N H7L H7M H7J H7K H7H H7G H7E H7B H7C H7A H7...,1124922301
3,Gatineau,Gatineau,QC,Quebec,45.4833,-75.6500,276245.0,773.7,America/Montreal,2,J8P J8R J8T J8Y J8X J8Z J8M J9J J9H J9A,1124722129
4,Longueuil,Longueuil,QC,Quebec,45.5333,-73.5167,239700.0,2002.0,America/Montreal,2,J4T J4V J4P J4R J4M J4L J4N J4H J4K J4J J4G J3...,1124122753
...,...,...,...,...,...,...,...,...,...,...,...,...
634,Saint-Bonaventure,Saint-Bonaventure,QC,Quebec,45.9667,-72.6833,1017.0,12.9,America/Montreal,4,J0C,1124324069
635,Brébeuf,Brebeuf,QC,Quebec,46.0667,-74.6667,1012.0,27.9,America/Montreal,4,J0T,1124001084
636,Baie-du-Febvre,Baie-du-Febvre,QC,Quebec,46.1300,-72.7200,1010.0,10.4,America/Montreal,3,J0G,1124218916
637,Durham-Sud,Durham-Sud,QC,Quebec,45.6667,-72.3333,1008.0,10.8,America/Montreal,4,J0H,1124105436


In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list_qc=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url_qc = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results_qc = rq.get(url_qc).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list_qc.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_qc])

    nearby_venues_qc = pd.DataFrame([item for venues_list_qc in venues_list_qc for item in venues_list_qc])
    nearby_venues_qc.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_qc)

In [154]:
cnd_qc_mon=cnd_qc[cnd_qc['city']=='Montréal']

In [155]:
qc_venues = getNearbyVenues(names= cnd_qc_mon['city'],
                                   latitudes=cnd_qc_mon['lat'],
                                   longitudes=cnd_qc_mon['lng']
                                  )

Montréal


In [156]:
print(qc_venues.shape)
qc_venues.head()

(42, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Montréal,45.5089,-73.5617,Cachitos,45.509814,-73.564077,Bakery
1,Montréal,45.5089,-73.5617,Le Mal Nécessaire,45.508536,-73.561218,Cocktail Bar
2,Montréal,45.5089,-73.5617,Société des arts technologiques [SAT],45.509369,-73.562654,Music Venue
3,Montréal,45.5089,-73.5617,UnderDog Boxing Gym,45.510055,-73.563710,Boxing Gym
4,Montréal,45.5089,-73.5617,SAT FoodLab,45.509446,-73.562668,Restaurant


In [157]:
print('There are {} uniques categories.'.format(len(qc_venues['Venue Category'].unique())))

There are 28 uniques categories.


In [166]:
# one hot encoding
qc_onehot = pd.get_dummies(qc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
qc_onehot['Neighborhood'] = qc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [qc_onehot.columns[-1]] + list(qc_onehot.columns[:-1])
qc_onehot = qc_onehot[fixed_columns]

qc_onehot.head()

,Vietnamese Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Boxing Gym,Café,Chinese Restaurant,Cocktail Bar,Comedy Club,Concert Hall,Department Store,Dessert Shop,Food Court,French Restaurant,Gastropub,Hotel,Italian Restaurant,Japanese Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Plaza,Public Art,Restaurant,Sports Bar,Taco Place,Vegetarian / Vegan Restaurant
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Montréal,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,1,0,0,0


In [159]:
qc_onehot.shape
qc_onehot.info()
qc_onehot['Home Prices'] ="0"
qc_onehot_home_price = qc_onehot

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Vietnamese Restaurant          42 non-null     uint8 
 1   Art Gallery                    42 non-null     uint8 
 2   Art Museum                     42 non-null     uint8 
 3   Asian Restaurant               42 non-null     uint8 
 4   Bakery                         42 non-null     uint8 
 5   Boxing Gym                     42 non-null     uint8 
 6   Café                           42 non-null     uint8 
 7   Chinese Restaurant             42 non-null     uint8 
 8   Cocktail Bar                   42 non-null     uint8 
 9   Comedy Club                    42 non-null     uint8 
 10  Concert Hall                   42 non-null     uint8 
 11  Department Store               42 non-null     uint8 
 12  Dessert Shop                   42 non-null     uint8 
 13  Food Co

In [160]:
qc_onehot_home_price.head()


,Vietnamese Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Boxing Gym,Café,Chinese Restaurant,Cocktail Bar,Comedy Club,Concert Hall,Department Store,Dessert Shop,Food Court,French Restaurant,Gastropub,Hotel,Italian Restaurant,Japanese Restaurant,Music Venue,Neighborhood,Performing Arts Venue,Plaza,Public Art,Restaurant,Sports Bar,Taco Place,Vegetarian / Vegan Restaurant,Home Prices
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Montréal,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,1,0,0,0,0


In [161]:
qc_grouped = qc_onehot.groupby('Neighborhood').mean().reset_index()
qc_grouped

,Neighborhood,Vietnamese Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Boxing Gym,Café,Chinese Restaurant,Cocktail Bar,Comedy Club,Concert Hall,Department Store,Dessert Shop,Food Court,French Restaurant,Gastropub,Hotel,Italian Restaurant,Japanese Restaurant,Music Venue,Performing Arts Venue,Plaza,Public Art,Restaurant,Sports Bar,Taco Place,Vegetarian / Vegan Restaurant
0,Montréal,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.047619,0.02381,0.095238,0.02381,0.02381,0.02381,0.047619,0.02381,0.071429,0.02381,0.02381,0.02381,0.047619,0.047619,0.02381,0.095238,0.02381,0.02381,0.02381


In [162]:
feat=[]
for x in qc_onehot[:0]:
  feat.append(x)
print(feat)

['Vietnamese Restaurant', 'Art Gallery', 'Art Museum', 'Asian Restaurant', 'Bakery', 'Boxing Gym', 'Café', 'Chinese Restaurant', 'Cocktail Bar', 'Comedy Club', 'Concert Hall', 'Department Store', 'Dessert Shop', 'Food Court', 'French Restaurant', 'Gastropub', 'Hotel', 'Italian Restaurant', 'Japanese Restaurant', 'Music Venue', 'Neighborhood', 'Performing Arts Venue', 'Plaza', 'Public Art', 'Restaurant', 'Sports Bar', 'Taco Place', 'Vegetarian / Vegan Restaurant', 'Home Prices']


### Methodlogy Section
I used Toronto venues and Home Prices to build Multiple linear regression model for predicting home prices in Montreal.

In [167]:
toronto_onehot_home_price[feat]

KeyError: ignored

In [168]:
missing = ['Sports Bar', 'Comedy Club', 'Public Art', 'Boxing Gym', 'Food Court', 'Taco Place', 'Plaza', 'Art Museum', 'Gastropub', 'Music Venue', 'Concert Hall']

In [169]:
for i in missing:
  feat.remove(i)
  #print(i)
feat

['Vietnamese Restaurant',
 'Art Gallery',
 'Asian Restaurant',
 'Bakery',
 'Café',
 'Chinese Restaurant',
 'Cocktail Bar',
 'Department Store',
 'Dessert Shop',
 'French Restaurant',
 'Hotel',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Neighborhood',
 'Performing Arts Venue',
 'Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Home Prices']

In [170]:
Feature = qc_onehot_home_price[feat]

Feature.head()


,Vietnamese Restaurant,Art Gallery,Asian Restaurant,Bakery,Café,Chinese Restaurant,Cocktail Bar,Department Store,Dessert Shop,French Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Neighborhood,Performing Arts Venue,Restaurant,Vegetarian / Vegan Restaurant,Home Prices
0,0,0,0,1,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,Montréal,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,Montréal,0,1,0,0


In [171]:
Feature=toronto_onehot_home_price[feat].drop('Neighborhood',axis=1)
Feature.head()

,Vietnamese Restaurant,Art Gallery,Asian Restaurant,Bakery,Café,Chinese Restaurant,Cocktail Bar,Department Store,Dessert Shop,French Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Performing Arts Venue,Restaurant,Vegetarian / Vegan Restaurant,Home Prices
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,484444.0


In [134]:
feat

['Vietnamese Restaurant',
 'Art Gallery',
 'Art Museum',
 'Asian Restaurant',
 'Bakery',
 'Café',
 'Chinese Restaurant',
 'Cocktail Bar',
 'Concert Hall',
 'Department Store',
 'Dessert Shop',
 'French Restaurant',
 'Gastropub',
 'Hotel',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Music Venue',
 'Neighborhood',
 'Performing Arts Venue',
 'Plaza',
 'Restaurant',
 'Taco Place',
 'Vegetarian / Vegan Restaurant',
 'Home Prices']

In [97]:
x_real =['Vietnamese Restaurant',
 'Art Gallery',
 'Bakery',
 'Café',
 'Chinese Restaurant',
 'Cocktail Bar',
 'Department Store',
 'Dessert Shop',
 'French Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Performing Arts Venue',
 'Restaurant',
 'Vegetarian / Vegan Restaurant']

In [172]:
msk = np.random.rand(len(Feature)) < 0.8
train = Feature[msk]
test = Feature[~msk]

I randomly select data for train and test model!

In [173]:
train.head()

,Vietnamese Restaurant,Art Gallery,Asian Restaurant,Bakery,Café,Chinese Restaurant,Cocktail Bar,Department Store,Dessert Shop,French Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Performing Arts Venue,Restaurant,Vegetarian / Vegan Restaurant,Home Prices
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,484444.0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,484444.0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,484444.0


In [174]:
test.head()

,Vietnamese Restaurant,Art Gallery,Asian Restaurant,Bakery,Café,Chinese Restaurant,Cocktail Bar,Department Store,Dessert Shop,French Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Performing Arts Venue,Restaurant,Vegetarian / Vegan Restaurant,Home Prices
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,365107.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,484444.0
15,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,484444.0
22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,484444.0


In [175]:
X = train[x_real]
X = np.asarray(train[x_real])
X[0:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [176]:
X= preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

array([[-0.13693064, -0.11145564, -0.15861032, -0.19549056, -0.07856742,
        -0.07856742, -0.07856742, -0.07856742, -0.11145564, -0.07856742,
         0.        ,  0.        , -0.21182964, -0.11145564],
       [-0.13693064, -0.11145564, -0.15861032, -0.19549056, -0.07856742,
        -0.07856742, -0.07856742, -0.07856742, -0.11145564, -0.07856742,
         0.        ,  0.        , -0.21182964, -0.11145564],
       [-0.13693064, -0.11145564,  6.30476011, -0.19549056, -0.07856742,
        -0.07856742, -0.07856742, -0.07856742, -0.11145564, -0.07856742,
         0.        ,  0.        , -0.21182964, -0.11145564],
       [-0.13693064, -0.11145564, -0.15861032, -0.19549056, -0.07856742,
        -0.07856742, -0.07856742, -0.07856742, -0.11145564, -0.07856742,
         0.        ,  0.        , -0.21182964, -0.11145564],
       [-0.13693064, -0.11145564, -0.15861032, -0.19549056, -0.07856742,
        -0.07856742, -0.07856742, -0.07856742, -0.11145564, -0.07856742,
         0.        ,  0.  

In [177]:
y=train['Home Prices']
y=np.asarray(train['Home Prices'])
y[0:5]

array([365107., 365107., 484444., 484444., 484444.])

In [178]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
xx = np.asanyarray(train[x_real])
yy = np.asanyarray(train[['Home Prices']])
regr.fit (xx, yy)
# The coefficients
print ('Coefficients: ', regr.coef_)

Coefficients:  [[ -36620.1969697    26712.13636364    5545.88636364   25794.8030303
  -193800.86363636   41970.13636364 -179389.86363636   11454.13636364
    26712.13636364   41970.13636364       0.               0.
    -1278.57792208   41970.13636364]]


In [179]:
X1=test[x_real]
X1= preprocessing.StandardScaler().fit(X1).transform(X1.astype(float))
X1[0:5]

array([[ 0.        ,  0.        ,  0.        ,  0.        , -0.15075567,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.21566555, -0.15075567,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.15075567,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.21566555, -0.15075567,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.15075567,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.21566555, -0.15075567,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.15075567,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.21566555,  6.63324958,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.15075567,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.21566555, -0.15

In [180]:
X1 = np.asarray(test[x_real])
X1[0:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [181]:
y_hat= regr.predict(X1)
x = np.asanyarray(test[x_real])
y = np.asanyarray(test[['Home Prices']])
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

 #Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

Residual sum of squares: 10756956191.57
Variance score: -0.15


### Analysis Discussion

So, This model is not looking good. It has a Variance score : -0.15 ,but for the perfect variance score is 1.

In [184]:
Feature=qc_onehot_home_price[x_real]
Feature.head()

,Vietnamese Restaurant,Art Gallery,Bakery,Café,Chinese Restaurant,Cocktail Bar,Department Store,Dessert Shop,French Restaurant,Italian Restaurant,Japanese Restaurant,Performing Arts Venue,Restaurant,Vegetarian / Vegan Restaurant
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [186]:
XX = np.asarray(Feature)

### Conclusion Section

Finally, I have Montreal Home Price result!!!

In [187]:
housing_price = regr.predict(XX)
housing_price

array([[478535.75      ],
       [514960.        ],
       [472989.86363636],
       [472989.86363636],
       [471711.28571429],
       [471711.28571429],
       [472989.86363636],
       [472989.86363636],
       [472989.86363636],
       [293600.        ],
       [472989.86363636],
       [279189.        ],
       [472989.86363636],
       [499702.        ],
       [472989.86363636],
       [472989.86363636],
       [472989.86363636],
       [471711.28571429],
       [472989.86363636],
       [472989.86363636],
       [484444.        ],
       [472989.86363636],
       [472989.86363636],
       [499702.        ],
       [498784.66666667],
       [499702.        ],
       [472989.86363636],
       [472989.86363636],
       [514960.        ],
       [472989.86363636],
       [498784.66666667],
       [472989.86363636],
       [471711.28571429],
       [472989.86363636],
       [472989.86363636],
       [472989.86363636],
       [472989.86363636],
       [436369.66666667],
       [5149